In [1]:
import time
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
%load_ext autoreload
%autoreload 2

In [60]:
from intabs_multi.dataset import InnDataSet
data_name = "housing"
d = InnDataSet(data_name)

In [53]:
from intabs_multi.classifier_multi import *
num_h_neurons = 15
epochs = 40
cross_validation(d, num_h_neurons, epochs, data_name)

Using cuda device
accuracy: 0.695126715816133+-0.022040635782659964


In [54]:
num_h_neurons = 15
epochs = 40
torch_model = train_clf(d.X1_train, d.y1_train, d.X1_test, d.y1_test, num_h_neurons, epochs, data_name)
model = InnModel(d, torch_model, num_h_neurons)

Evaluations on training data

               precision    recall  f1-score   support

         (0)      0.741     0.830     0.783      2722
         (1)      0.600     0.482     0.535      2738
         (2)      0.718     0.772     0.744      2795

    accuracy                          0.695      8255
   macro avg      0.686     0.695     0.687      8255
weighted avg      0.686     0.695     0.687      8255

Evaluations on testing data

               precision    recall  f1-score   support

         (0)      0.735     0.829     0.780       680
         (1)      0.613     0.480     0.538       685
         (2)      0.734     0.798     0.765       699

    accuracy                          0.703      2064
   macro avg      0.694     0.703     0.694      2064
weighted avg      0.694     0.703     0.695      2064



In [55]:
rt_models, val_set, val_y_set = get_retrained_models_and_validation_set(d, model)

100%|██████████| 5/5 [00:22<00:00,  4.52s/it]


In [58]:
print(val_set)

[[1.98280024e-01 7.45098039e-01 3.67273877e-02 1.97628458e-02
  2.49446453e-03 1.44384320e-03 5.49415515e-01 3.26693227e-01]
 [1.10384684e-01 6.66666667e-01 1.98974117e-02 2.13322483e-02
  6.36508871e-02 3.03105705e-03 1.51965994e-01 6.11553785e-01]
 [1.08970911e-01 5.68627451e-01 3.04414643e-02 2.14032402e-02
  4.76190476e-02 2.14971872e-03 6.99256111e-01 2.76892430e-01]
 [1.87114660e-01 5.88235294e-02 2.50054352e-02 2.42273442e-02
  3.49505311e-02 9.57816782e-04 1.60467588e-01 7.11155378e-01]
 [1.02219280e-01 4.31372549e-01 2.80698056e-02 2.66914423e-02
  9.13983015e-02 2.24614357e-03 1.61530287e-01 7.07171315e-01]
 [1.05750266e-01 2.15686275e-01 3.07295355e-02 2.56916996e-02
  3.64359988e-02 2.77167756e-03 3.88947928e-01 4.99003984e-01]
 [1.77314796e-01 1.76470588e-01 4.14095033e-02 2.33406024e-02
  3.58754449e-02 1.66804288e-03 2.11477152e-01 7.06175299e-01]
 [2.22707273e-01 7.05882353e-01 4.83107983e-02 4.48851075e-02
  5.07301214e-03 1.95256420e-03 1.68969182e-01 8.04780876e-01]


In [64]:
from intabs_multi.methods_multi import rnce
from intabs_multi.util import eval_empirical_robustness, get_test_inputs
val_delta = 0.04
ces = rnce(val_set, d, model, delta=val_delta, y_target=2)
print(eval_empirical_robustness(rt_models, ces, y_target=2))

7480it [02:25, 51.27it/s]

True


In [65]:
# val delta
test_set, test_set_df, test_set_full_df = get_test_inputs(d, model, num_inputs=20)

In [66]:
from intabs_multi.classifier_multi import get_retrained_models_all
rt_models_eval = get_retrained_models_all(d, model, num_h_neurons, epochs)

100%|██████████| 5/5 [00:46<00:00,  9.37s/it]


In [67]:
from intabs_multi.methods_multi import run_all
run_all(d, model, rt_models_eval, val_delta)

7480it [02:34, 48.38it/s]
20it [00:10,  1.92it/s]
7480it [02:31, 49.50it/s]
20it [00:09,  2.18it/s]
7480it [02:26, 51.18it/s]
20it [00:09,  2.19it/s]
7480it [02:26, 51.01it/s]
20it [00:09,  2.15it/s]
7480it [02:37, 47.45it/s]
20it [00:09,  2.08it/s]


average results
+----------+------------+--------+-------+-------+--------------+
| name     |   coverage |   cost |   lof |   vm2 |   vdelta-val |
+==========+============+========+=======+=======+==============+
| nnce     |          1 |  0.032 | 1.098 | 0.181 |            0 |
| rnce     |          1 |  0.053 | 1.331 | 1     |            1 |
| rnce-opt |          1 |  0.052 | 1.307 | 1     |            1 |
+----------+------------+--------+-------+-------+--------------+
std results
+----------+------------+--------+-------+-------+--------------+
| name     |   coverage |   cost |   lof |   vm2 |   vdelta-val |
+==========+============+========+=======+=======+==============+
| nnce     |          0 |  0.003 | 0.02  | 0.035 |            0 |
| rnce     |          0 |  0.004 | 0.037 | 0     |            0 |
| rnce-opt |          0 |  0.004 | 0.035 | 0     |            0 |
+----------+------------+--------+-------+-------+--------------+
